### data load & Setting

In [1]:
# For Colab Use
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Package loading
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [3]:
# data load for regression
data = pd.read_excel('/content/drive/MyDrive/SP/D_dong_Dataset_netcard영향14개구는 구별 가정으로포함.xlsx', sheet_name = "dong_wk20_35")

In [ ]:
# data to calculate X vars

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6784 entries, 0 to 6783
Data columns (total 37 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   id                                         6784 non-null   object 
 1   adong_cd                                   6784 non-null   int64  
 2   ADM_DR_C                                   6784 non-null   int64  
 3   ADM_DR_CD                                  6784 non-null   int64  
 4   realcase                                   6784 non-null   object 
 5   week                                       6784 non-null   int64  
 6   m1wk_ago                                   6784 non-null   int64  
 7   m2wk_ago                                   6784 non-null   int64  
 8   if11gu                                     6784 non-null   int64  
 9   gu                                         6784 non-null   object 
 10  id_gu                   

In [5]:
data.head()

,id,adong_cd,ADM_DR_C,ADM_DR_CD,realcase,week,m1wk_ago,m2wk_ago,if11gu,gu,...,zero_replace_available,sigSWM_m1wkxinf_m1wk,sigSWM_m1wkxinf_m1wk_nonSeoul,sigSWM_m1wkxinf_m1wk_Seoul,signetCard_samexorig_inf_m1wk,signetCard_diffxorig_infm1wk_frSeoul,signetCard_diffxorig_infm1wk_fromNonSeoul,signetCard_diffxorig_infm1wk_fromKorea,Exposure_sum_netCardxdest_inf,Infm2wk_orig_assumption
0,20송파구가락1동,1171063100,1171063,1124066,.,20,19,18,0,송파구,...,0,212111.465553,14031.322152,198080.143400,26.597876,-367.904947,4.725553,-363.179394,828.319921,0.148384
1,20송파구가락2동,1171063200,1171063,1124067,.,20,19,18,0,송파구,...,0,293913.237671,6373.706949,287539.530722,-211.336259,222.410231,4.449058,226.859289,1471.827421,0.148384
2,20송파구가락본동,1171062000,1171062,1124065,.,20,19,18,0,송파구,...,0,338605.747418,15812.542589,322793.204829,154.022845,277.345704,6.208572,283.554276,980.399167,0.148384
3,20구로구가리봉동,1153059500,1153059,1117070,.,20,19,18,0,구로구,...,0,30048.536797,6696.056614,23352.480183,0.000000,34.145702,0.000000,34.145702,511.797492,0.000000
4,20금천구가산동,1154551000,1154551,1118051,.,20,19,18,0,금천구,...,0,428729.917524,121958.857617,306771.059907,232.715315,1134.009341,516.306926,1650.316267,1033.917164,0.000000


### OLS Regression with prepared data

In [6]:
# Adjust the decimal points of the X variables
data['signetCardxinf_m1wk_same_de4'] = data['signetCard_samexorig_inf_m1wk'] / 10000
data['sigSWM_m1wkxinf_m1wk_de6'] = data['sigSWM_m1wkxinf_m1wk'] / 1000000
data['signetCardxinf_m1wk_diff_de4'] = data['signetCard_diffxorig_infm1wk_fromKorea'] / 10000
data['Exposu_netCardxdestinf_wkm1_de3'] = data['Exposure_sum_netCardxdest_inf'] / 1000

# Define the list of gu values to filter
gu_values = ["강북구", "관악구", "노원구", "동대문구", "동작구", "양천구", "영등포구", "용산구", "은평구", "종로구", "중랑구"]

# Filter the data to include only rows where 'gu' is in gu_values
filtered_data = data[data['gu'].isin(gu_values)]

# Define the explanatory variables (X) and the dependent variable (y) using the filtered dataset
X = filtered_data[['Infm2wk_orig_assumption',
                   'sigSWM_m1wkxinf_m1wk_de6',
                   'signetCardxinf_m1wk_same_de4',
                   'signetCardxinf_m1wk_diff_de4',
                   'Exposu_netCardxdestinf_wkm1_de3']]
y = filtered_data['Inf_orig_assumption']

In [ ]:
# # Define the explanatory variables (X) and the dependent variable (y) using the whole dataset
# X = data[['Infm2wk_orig_assumption',
#           'sigSWM_m1wkxinf_m1wk',
#           'signetCard_samexorig_inf_m1wk',
#           'signetCard_diffxorig_infm1wk_fromKorea',
#           'Exposure_sum_netCardxdest_inf']]
# y = data['Inf_orig_assumption']

In [56]:
# Run the OLS regression model without intercept
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())

                                 OLS Regression Results                                 
Dep. Variable:     Inf_orig_assumption   R-squared (uncentered):                   0.267
Model:                             OLS   Adj. R-squared (uncentered):              0.265
Method:                  Least Squares   F-statistic:                              212.5
Date:                 Thu, 03 Oct 2024   Prob (F-statistic):                   8.78e-194
Time:                         12:12:15   Log-Likelihood:                         -8487.9
No. Observations:                 2928   AIC:                                  1.699e+04
Df Residuals:                     2923   BIC:                                  1.702e+04
Df Model:                            5                                                  
Covariance Type:             nonrobust                                                  
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------

### Simulation - 지역제한 있을 때

In [57]:
# Prepare empty DataFrames for predictions (시뮬레이션 결과를 넣을 데이터셋 생성)
weeks = range(20, 36)
dongs = data['dong'].unique() # 424개 동, 11개구 내 속한 동으로만 하려면 dongs = filtered_data['dong'].unique()로 해야함
predictions_df = pd.DataFrame(index=dongs, columns=[f"week_{week}" for week in weeks])

In [58]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 송파구가락1동 to 동작구흑석동
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   week_20  0 non-null      object
 1   week_21  0 non-null      object
 2   week_22  0 non-null      object
 3   week_23  0 non-null      object
 4   week_24  0 non-null      object
 5   week_25  0 non-null      object
 6   week_26  0 non-null      object
 7   week_27  0 non-null      object
 8   week_28  0 non-null      object
 9   week_29  0 non-null      object
 10  week_30  0 non-null      object
 11  week_31  0 non-null      object
 12  week_32  0 non-null      object
 13  week_33  0 non-null      object
 14  week_34  0 non-null      object
 15  week_35  0 non-null      object
dtypes: object(16)
memory usage: 56.3+ KB


In [59]:
predictions_df.head(3)

,week_20,week_21,week_22,week_23,week_24,week_25,week_26,week_27,week_28,week_29,week_30,week_31,week_32,week_33,week_34,week_35
송파구가락1동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
송파구가락2동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
송파구가락본동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### SWMxinf 관련 변수 계산방법

In [61]:
# SWMxinf_S
SWMxinf_S = pd.read_excel('/content/drive/MyDrive/SP/SWMxinf_from_Seoul.xlsx') # 유동인구 from Seoul to Seoul
population = pd.read_excel('/content/drive/MyDrive/SP/population_by_gu_dong.xlsx')
# SWMxinf_NS
SWMxinf_NS = pd.read_excel('/content/drive/MyDrive/SP/SWMxinf_from_nonSeoul.xlsx') # 유동인구 from 비서울 to Seoul

In [15]:
SWMxinf_S.head()

,O_D,week,week_orig,orig_gu,week_dest,avg_flow_10to17,inforig,avg_SWMxinforig,dest
0,종로구_종로구청운효자동,19,19종로구,종로구,19종로구청운효자동,102960.709426,0.0,0.0,종로구청운효자동
1,종로구_종로구사직동,19,19종로구,종로구,19종로구사직동,92704.055876,0.0,0.0,종로구사직동
2,종로구_종로구삼청동,19,19종로구,종로구,19종로구삼청동,24142.027647,0.0,0.0,종로구삼청동
3,종로구_종로구부암동,19,19종로구,종로구,19종로구부암동,87847.131278,0.0,0.0,종로구부암동
4,종로구_종로구평창동,19,19종로구,종로구,19종로구평창동,141354.636560,0.0,0.0,종로구평창동


In [35]:
population.head()

,gu,dong,population
0,강북구,강북구미아동,22723
1,강북구,강북구번1동,19441
2,강북구,강북구번2동,16544
3,강북구,강북구번3동,17605
4,강북구,강북구삼각산동,32118


 #### netCard x inf 관련 변수 계산

In [17]:
Card_S = pd.read_excel('/content/drive/MyDrive/SP/netCard_from_Seoul.xlsx') # Card from Seoul to Seoul
Card_NS = pd.read_excel('/content/drive/MyDrive/SP/netCard_from_nonSeoul.xlsx') # Card from 비서울 to Seoul
Card_S.head()

,cat_origin,week,id_week_orig_dong,id_week_dest_dong,num_total,num_stmchk,avg_num_total,netCard,sameOD,orig_dong_name,orig_inf,netCardxorig_inf,dest_dong_name,dest_inf,netCardxdest_inf,Iforig_gu11,Ifdest_gu11
0,서울,19,19강남구개포1동,19강남구개포1동,169,0,227.375,-58.375,1,강남구개포1동,1.10283,-64.377682,강남구개포1동,1.10283,-64.377682,0,0
1,서울,19,19강남구개포1동,19강남구개포2동,44,0,49.875,-5.875,0,강남구개포1동,1.10283,-6.479124,강남구개포2동,1.10283,-6.479124,0,0
2,서울,19,19강남구개포1동,19강남구개포4동,38,0,27.625,10.375,0,강남구개포1동,1.10283,11.441858,강남구개포4동,1.10283,11.441858,0,0
3,서울,19,19강남구개포1동,19강남구대치1동,27,0,9.500,17.500,0,강남구개포1동,1.10283,19.299519,강남구대치1동,1.10283,19.299519,0,0
4,서울,19,19강남구개포1동,19강남구대치4동,9,0,6.875,2.125,0,강남구개포1동,1.10283,2.343513,강남구대치4동,1.10283,2.343513,0,0


In [64]:
# Function to fetch m2wk infection rates from 'data'
def get_m2wk_infection_rate(data, week, dong):
    # Generate the id for the target week
    target_id = f"{week}{dong}"

    # Filter the row with the corresponding id
    row = data[data['id'] == target_id]

    # Return the infection rate (Infm2wk_orig_assumption) if the row exists
    if not row.empty:
        return row['Infm2wk_orig_assumption'].values[0]
    else:
        return np.nan  # or any default value you prefer

In [65]:
# Iterate over each week to predict infection rates
for week in range(20, 36): # weeks = range(20, 36)
    # Create a DataFrame for the current week's prediction input
    df_input = pd.DataFrame(index=dongs) # 424개 동을 index로 갖는 데이터프레임 만듦
    df_input['week_dest'] = [f"{week}{dong}" for dong in df_input.index]
    df_input['m1wk_dest'] = [f"{week - 1}{dong}" for dong in df_input.index]

    # Calculate explanatory variables for the current week
    # 1) 2주 전 해당 지역의 감염률 관련변수: Add 'Infm2wk_orig_assumption'
    if week in [20, 21]:
       df_input['Infm2wk_orig_assumption'] = df_input.index.map(lambda dong: get_m2wk_infection_rate(data, week, dong))
    else:
       df_input['Infm2wk_orig_assumption'] = df_input.index.map(lambda dong: predictions_df.at[dong, f'week_{week - 2}'])

    # 2) 유동인구패턴 관련변수: Add sigSWM_m1wkxinf_m1wk_de6
        # (aggreated_df에서 'sigSWM_m1wkxinf_m1wk_de6'값 각각을 df_input의 week_dest가 일치하는 행에 찾아넣기)
    if week == 20: # 19주차에 해당하는 변수를 가져와야함
       # Group by the 'week_dest' column and sum the 'avg_SWMxinforig' values
       aggregated_SWM = SWMxinf_S.groupby('week_dest')['avg_SWMxinforig'].sum().reset_index()
    else: # if week >= 21
       # 구별 infections rate 계산: 동별 predicted values의 population 가중 평균
       # Merge predictions_df with population_data to get the population and gu
       merged_pop_inf = pd.merge(population, predictions_df[[f'week_{week - 1}']], left_on='dong', right_index=True)

       # Calculate total infections for each dong
       merged_pop_inf['infectionsxpopulation'] = merged_pop_inf[f'week_{week -1}'] * merged_pop_inf['population']

       # Aggregate total infections and population by gu
       gu_infections = merged_pop_inf.groupby('gu')['infectionsxpopulation'].sum().reset_index()
       gu_population = merged_pop_inf.groupby('gu')['population'].sum().reset_index()

       # Merge the total infections and population dataframes
       gu_merged = pd.merge(gu_infections, gu_population, on='gu')

       # Calculate the infection rate for each gu
       gu_merged['infection_rate'] = gu_merged['infectionsxpopulation'] / gu_merged['population']

       # Filter rows in SWMxinf_S where the week column matches the previous week value
       filtered_SWMxinf_S = SWMxinf_S[SWMxinf_S['week'] == (week - 1)]

       # Merge the filtered DataFrame with gu_merged based on 'orig_gu' and 'gu'
       merged_df = pd.merge(filtered_SWMxinf_S, gu_merged, left_on='orig_gu', right_on='gu', how='left')

       # Update the inforig column in the original SWMxinf_S DataFrame
       SWMxinf_S.loc[SWMxinf_S['week'] == (week-1), 'inforig'] = merged_df['infection_rate'].values

       # SWMxinf_S의 avg_SWMxinforig 계산(현재 week 해당분만)
       mask = SWMxinf_S['week'] == (week-1)
       SWMxinf_S.loc[mask, 'avg_SWMxinforig'] = SWMxinf_S.loc[mask, 'avg_flow_10to17'] * SWMxinf_S.loc[mask, 'inforig']

       # Group by the 'week_dest' and calculate sum the 'avg_SWMxinforig' values
       aggregated_SWM = SWMxinf_S.groupby('week_dest')['avg_SWMxinforig'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum, if desired
    aggregated_SWM.rename(columns={'avg_SWMxinforig': 'sum_avg_SWMxinforig'}, inplace=True)

    aggregated_SWM['sigSWM_m1wkxinf_m1wk_de6'] = aggregated_SWM['sum_avg_SWMxinforig'] / 1000000

    df_input = pd.merge(df_input, aggregated_SWM[['week_dest','sigSWM_m1wkxinf_m1wk_de6']], how = 'left', left_on = 'm1wk_dest', right_on = 'week_dest')

   # 3) 같은 OD간 카드사용 변화 관련 변수: 'signetCardxinf_m1wk_same_de4'
    if week == 20: # 20주차 estimate 위한 변수로는 19주차에 해당하는 변수를 가져옴
       # sameOD_mask
       mask_sameOD = Card_S['orig_dong_name'] == Card_S['dest_dong_name']

       # sameOD netCardxorig_inf 계산
       Card_S.loc[mask_sameOD, 'netCardxorig_inf'] = Card_S.loc[mask_sameOD, 'netCard'] * Card_S.loc[mask_sameOD, 'orig_inf']

       # Group by the 'id_week_dest_dong' and calculate sum the 'netCard_samexinforig' values
       aggregated_Card_S_sameOD = Card_S[mask_sameOD].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    else: # this part not yet coded
       # Filter rows in netCardxinf where the week column matches the previous week value
       filtered_Card_S = Card_S[(Card_S['week'] == (week - 1))]
       # predictions_df에서 해당 주차의 감염률 데이터를 가져옴
       m1wk_column = f'week_{week - 1}'

       # Update the Card_S['orig_inf'] from the predictions_df dataframe's week_{week - 1} column
       # (orig_dong_name과 predictions_df의 index(동 이름)가 일치하는 행을 찾아 orig_inf 값을 업데이트)
       filtered_Card_S['orig_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['orig_dong_name'], m1wk_column]
                    if row['orig_dong_name'] in predictions_df.index else row['orig_inf'], axis=1)

       # Update netCardxorig_inf
       filtered_Card_S['netCardxorig_inf'] = filtered_Card_S['netCard'] * filtered_Card_S['orig_inf']

       # sameOD_mask
       mask_sameOD_in_filtered = filtered_Card_S['orig_dong_name'] == filtered_Card_S['dest_dong_name']

       # aggregate netCardxorig_inf by destination dong
       aggregated_Card_S_sameOD = filtered_Card_S[mask_sameOD_in_filtered].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum
    aggregated_Card_S_sameOD.rename(columns={'netCardxorig_inf': 'signetCardxinf_same'}, inplace=True)
    # Create a decimal point adjusted-variable
    aggregated_Card_S_sameOD['signetCardxinf_same_de4'] = aggregated_Card_S_sameOD['signetCardxinf_same'] / 10000

    # merge signetCardxinf_same_de4 based on destination dong
    df_input = pd.merge(df_input, aggregated_Card_S_sameOD[['id_week_dest_dong', 'signetCardxinf_same_de4']],
                  left_on='m1wk_dest', right_on='id_week_dest_dong', how='left')
    df_input['signetCardxinf_m1wk_same_de4'] = df_input['signetCardxinf_same_de4']
    df_input = df_input.drop(['id_week_dest_dong', 'signetCardxinf_same_de4'], axis=1)


   # 4) 다른 OD간 카드사용 변화 관련 변수1 (들어오는): 'signetCardxinf_m1wk_diff_de4'
    if week == 20:
      # diffOD_mask
       mask_diffOD = Card_S['orig_dong_name'] != Card_S['dest_dong_name']

      # diffOD netCardxorig_inf 계산
       Card_S.loc[mask_diffOD, 'netCardxorig_inf'] = Card_S.loc[mask_diffOD, 'netCard'] * Card_S.loc[mask_diffOD, 'orig_inf']

      # Group by the 'id_week_dest_dong' and calculate sum the 'netCard_diffxinforig' values
       aggregated_Card_S_diffOD = Card_S[mask_diffOD].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    else: # if week >= 21
       # diffOD_mask
       mask_diffOD_in_filtered = filtered_Card_S['orig_dong_name'] != filtered_Card_S['dest_dong_name']

       # aggregate netCardxorig_inf by destination dong
       aggregated_Card_S_diffOD = filtered_Card_S[mask_diffOD_in_filtered].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum
    aggregated_Card_S_diffOD.rename(columns={'netCardxorig_inf': 'signetCardxinf_diff'}, inplace=True)

    # Create a decimal point adjusted-variable
    aggregated_Card_S_diffOD['signetCardxinf_diff_de4'] = aggregated_Card_S_diffOD['signetCardxinf_diff'] / 10000

    # want to add 'signetCardxinf_m1wk_diff_de4' column in df_input dataframe according to m1wk_dest
    df_input = pd.merge(df_input, aggregated_Card_S_diffOD[['id_week_dest_dong', 'signetCardxinf_diff_de4']],
                  left_on='m1wk_dest', right_on='id_week_dest_dong', how='left')
    df_input['signetCardxinf_m1wk_diff_de4'] = df_input['signetCardxinf_diff_de4']
    df_input = df_input.drop(['id_week_dest_dong', 'signetCardxinf_diff_de4'], axis=1)


   # 5) 다른 OD간 카드사용 변화 관련 변수2 (나가는): 'Exposu_netCardxdestinf_wkm1_de3'
    if week == 20:
      # diffOD netCardxdest_inf 계산
      Card_S.loc[mask_diffOD, 'netCardxdest_inf'] = Card_S.loc[mask_diffOD, 'netCard'] * Card_S.loc[mask_diffOD, 'dest_inf']

      # Group by the 'id_week_orig_dong' and calculate sum of the 'netCard_diffxinfdest' values
      aggregated_Card_S_exposure = Card_S[mask_diffOD].groupby('id_week_orig_dong')['netCardxdest_inf'].sum().reset_index()
    else:
       # Update the Card_S['dest_inf'] from the predictions_df dataframe's week_{week - 1} column
       filtered_Card_S['dest_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['dest_dong_name'], m1wk_column]
                    if row['dest_dong_name'] in predictions_df.index else row['dest_inf'], axis=1)

       # Update netCardxdest_inf
       filtered_Card_S['netCardxdest_inf'] = filtered_Card_S['netCard'] * filtered_Card_S['dest_inf']

       # aggregate netCardxorig_inf by destination dong
       aggregated_Card_S_sameOD = filtered_Card_S[mask_diffOD_in_filtered].groupby('id_week_orig_dong')['netCardxdest_inf'].sum().reset_index()

    aggregated_Card_S_exposure.rename(columns = {'netCardxdest_inf': 'signetCardxdest_inf'}, inplace = True)
    aggregated_Card_S_exposure['signetCardxdest_inf_de3'] = aggregated_Card_S_exposure['signetCardxdest_inf']/1000

    df_input = pd.merge(df_input, aggregated_Card_S_exposure[['id_week_orig_dong', 'signetCardxdest_inf_de3']],
                  left_on = 'm1wk_dest', right_on = 'id_week_orig_dong', how = 'left')
    df_input['Exposu_netCardxdestinf_wkm1_de3'] = df_input['signetCardxdest_inf_de3']
    df_input = df_input.drop(['id_week_orig_dong', 'signetCardxdest_inf_de3'], axis = 1)

    # Save the current week's prediction input DataFrame to an Excel file
    df_input['dong_name'] = df_input.index
    df_input.to_excel(f'/content/drive/MyDrive/SP/input tables/df_input_week{week}.xlsx')

    # Predict the infection rates for the current week using the model
    X_input = df_input[['Infm2wk_orig_assumption',
                                  'sigSWM_m1wkxinf_m1wk_de6',
                                  'signetCardxinf_m1wk_same_de4',
                                  'signetCardxinf_m1wk_diff_de4',
                                  'Exposu_netCardxdestinf_wkm1_de3']]

    # Use the model to predict the infection rate
    y_pred = model.predict(X_input)

    # Replace negative values in y_pred with 0
    y_pred = np.where(y_pred < 0, 0, y_pred)

    # Update the predictions DataFrame with the predicted values
    predictions_df[f'week_{week}'] = y_pred

# Print the final predictions DataFrame
predictions_df.to_excel('predictions_df.xlsx', index=True)
print(predictions_df)

<ipython-input-65-64677ce3ff1d>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_Card_S['orig_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['orig_dong_name'], m1wk_column]
<ipython-input-65-64677ce3ff1d>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_Card_S['netCardxorig_inf'] = filtered_Card_S['netCard'] * filtered_Card_S['orig_inf']
<ipython-input-65-64677ce3ff1d>:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

           week_20   week_21   week_22   week_23   week_24   week_25  \
송파구가락1동   0.713043  0.451808  0.307212  0.216356  0.146584  0.101941   
송파구가락2동   1.135787  0.589708  0.423031  0.303071  0.219955  0.138450   
송파구가락본동   1.010740  0.736874  0.518097  0.377001  0.256190  0.178590   
구로구가리봉동   0.270650  0.217728  0.168335  0.144575  0.113749  0.088603   
금천구가산동    1.016545  1.102189  0.891799  0.769280  0.555648  0.423132   
...            ...       ...       ...       ...       ...       ...   
용산구효창동    0.549222  0.190680  0.123437  0.065865  0.042394  0.026400   
용산구후암동    1.463835  0.307952  0.244597  0.110221  0.067590  0.041684   
동대문구휘경1동  0.716863  0.514924  0.229090  0.172124  0.116783  0.083988   
동대문구휘경2동  0.959445  0.553779  0.333983  0.239343  0.173479  0.124429   
동작구흑석동    1.197715  0.897666  0.712544  0.554708  0.368574  0.270335   

           week_26   week_27   week_28   week_29   week_30   week_31  \
송파구가락1동   0.072593  0.050053  0.034816  0.023837  0.016677  0.0

In [67]:
# Function to fetch m2wk infection rates from 'data'
def get_m2wk_infection_rate(data, week, dong):
    # Generate the id for the target week
    target_id = f"{week}{dong}"

    # Filter the row with the corresponding id
    row = data[data['id'] == target_id]

    # Return the infection rate (Infm2wk_orig_assumption) if the row exists
    if not row.empty:
        return row['Infm2wk_orig_assumption'].values[0]
    else:
        return np.nan  # or any default value you prefer

# Iterate over each week to predict infection rates
for week in range(20, 36): # weeks = range(20, 36)
    # Create a DataFrame for the current week's prediction input
    df_input = pd.DataFrame(index=dongs) # 424개 동을 index로 갖는 데이터프레임 만듦
    df_input['week_dest'] = [f"{week}{dong}" for dong in df_input.index]
    df_input['m1wk_dest'] = [f"{week - 1}{dong}" for dong in df_input.index]

    # Calculate explanatory variables for the current week
    # 1) 2주 전 해당 지역의 감염률 관련변수: Add 'Infm2wk_orig_assumption'
    if week in [20, 21]:
        df_input['Infm2wk_orig_assumption'] = df_input.index.map(lambda dong: get_m2wk_infection_rate(data, week, dong))
    else:
        df_input['Infm2wk_orig_assumption'] = df_input.index.map(lambda dong: predictions_df.at[dong, f'week_{week - 2}'])

    # 2) 유동인구패턴 관련변수: Add sigSWM_m1wkxinf_m1wk_de6
    # (aggreated_df에서 'sigSWM_m1wkxinf_m1wk_de6'값 각각을 df_input의 week_dest가 일치하는 행에 찾아넣기)
    if week == 20: # 19주차에 해당하는 변수를 가져와야함
        # Group by the 'week_dest' column and sum the 'avg_SWMxinforig' values
        aggregated_SWM = SWMxinf_S.groupby('week_dest')['avg_SWMxinforig'].sum().reset_index()
    else: # if week >= 21
        # 구별 infections rate 계산: 동별 predicted values의 population 가중 평균
        # Merge predictions_df with population_data to get the population and gu
        merged_pop_inf = pd.merge(population, predictions_df[[f'week_{week - 1}']], left_on='dong', right_index=True)

        # Calculate total infections for each dong
        merged_pop_inf['infectionsxpopulation'] = merged_pop_inf[f'week_{week -1}'] * merged_pop_inf['population']

        # Aggregate total infections and population by gu
        gu_infections = merged_pop_inf.groupby('gu')['infectionsxpopulation'].sum().reset_index()
        gu_population = merged_pop_inf.groupby('gu')['population'].sum().reset_index()

        # Merge the total infections and population dataframes
        gu_merged = pd.merge(gu_infections, gu_population, on='gu')

        # Calculate the infection rate for each gu
        gu_merged['infection_rate'] = gu_merged['infectionsxpopulation'] / gu_merged['population']

        # Filter rows in SWMxinf_S where the week column matches the previous week value
        filtered_SWMxinf_S = SWMxinf_S[SWMxinf_S['week'] == (week - 1)]

        # Merge the filtered DataFrame with gu_merged based on 'orig_gu' and 'gu'
        merged_df = pd.merge(filtered_SWMxinf_S, gu_merged, left_on='orig_gu', right_on='gu', how='left')

        # Update the inforig column in the original SWMxinf_S DataFrame
        SWMxinf_S.loc[SWMxinf_S['week'] == (week-1), 'inforig'] = merged_df['infection_rate'].values

        # SWMxinf_S의 avg_SWMxinforig 계산(현재 week 해당분만)
        mask = SWMxinf_S['week'] == (week-1)
        SWMxinf_S.loc[mask, 'avg_SWMxinforig'] = SWMxinf_S.loc[mask, 'avg_flow_10to17'] * SWMxinf_S.loc[mask, 'inforig']

        # Group by the 'week_dest' and calculate sum the 'avg_SWMxinforig' values
        aggregated_SWM = SWMxinf_S.groupby('week_dest')['avg_SWMxinforig'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum, if desired
    aggregated_SWM.rename(columns={'avg_SWMxinforig': 'sum_avg_SWMxinforig'}, inplace=True)

    aggregated_SWM['sigSWM_m1wkxinf_m1wk_de6'] = aggregated_SWM['sum_avg_SWMxinforig'] / 1000000

    df_input = pd.merge(df_input, aggregated_SWM[['week_dest','sigSWM_m1wkxinf_m1wk_de6']], how='left', left_on='m1wk_dest', right_on='week_dest')

    # 3) 같은 OD간 카드사용 변화 관련 변수: 'signetCardxinf_m1wk_same_de4'
    if week == 20: # 20주차 estimate 위한 변수로는 19주차에 해당하는 변수를 가져옴
        # sameOD_mask
        mask_sameOD = Card_S['orig_dong_name'] == Card_S['dest_dong_name']

        # sameOD netCardxorig_inf 계산
        Card_S.loc[mask_sameOD, 'netCardxorig_inf'] = Card_S.loc[mask_sameOD, 'netCard'] * Card_S.loc[mask_sameOD, 'orig_inf']

        # Group by the 'id_week_dest_dong' and calculate sum the 'netCard_samexinforig' values
        aggregated_Card_S_sameOD = Card_S[mask_sameOD].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    else: # this part not yet coded
        # Filter rows in netCardxinf where the week column matches the previous week value
        filtered_Card_S = Card_S[(Card_S['week'] == (week - 1))]
        # predictions_df에서 해당 주차의 감염률 데이터를 가져옴
        m1wk_column = f'week_{week - 1}'

        # Update the Card_S['orig_inf'] from the predictions_df dataframe's week_{week - 1} column
        # (orig_dong_name과 predictions_df의 index(동 이름)가 일치하는 행을 찾아 orig_inf 값을 업데이트)
        filtered_Card_S.loc[:, 'orig_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['orig_dong_name'], m1wk_column]
            if row['orig_dong_name'] in predictions_df.index else row['orig_inf'], axis=1)

        # Update netCardxorig_inf
        filtered_Card_S.loc[:, 'netCardxorig_inf'] = filtered_Card_S['netCard'] * filtered_Card_S['orig_inf']

        # sameOD_mask
        mask_sameOD_in_filtered = filtered_Card_S['orig_dong_name'] == filtered_Card_S['dest_dong_name']

        # aggregate netCardxorig_inf by destination dong
        aggregated_Card_S_sameOD = filtered_Card_S[mask_sameOD_in_filtered].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum
    aggregated_Card_S_sameOD.rename(columns={'netCardxorig_inf': 'signetCardxinf_same'}, inplace=True)
    # Create a decimal point adjusted-variable
    aggregated_Card_S_sameOD['signetCardxinf_same_de4'] = aggregated_Card_S_sameOD['signetCardxinf_same'] / 10000

    # merge signetCardxinf_same_de4 based on destination dong
    df_input = pd.merge(df_input, aggregated_Card_S_sameOD[['id_week_dest_dong', 'signetCardxinf_same_de4']],
        left_on='m1wk_dest', right_on='id_week_dest_dong', how='left')
    df_input['signetCardxinf_m1wk_same_de4'] = df_input['signetCardxinf_same_de4']
    df_input = df_input.drop(['id_week_dest_dong', 'signetCardxinf_same_de4'], axis=1)

    # 4) 다른 OD간 카드사용 변화 관련 변수1 (들어오는): 'signetCardxinf_m1wk_diff_de4'
    if week == 20:
        # diffOD_mask
        mask_diffOD = Card_S['orig_dong_name'] != Card_S['dest_dong_name']

        # diffOD netCardxorig_inf 계산
        Card_S.loc[mask_diffOD, 'netCardxorig_inf'] = Card_S.loc[mask_diffOD, 'netCard'] * Card_S.loc[mask_diffOD, 'orig_inf']

        # Group by the 'id_week_dest_dong' and calculate sum the 'netCard_diffxinforig' values
        aggregated_Card_S_diffOD = Card_S[mask_diffOD].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    else: # if week >= 21
        # diffOD_mask
        mask_diffOD_in_filtered = filtered_Card_S['orig_dong_name'] != filtered_Card_S['dest_dong_name']

        # aggregate netCardxorig_inf by destination dong
        aggregated_Card_S_diffOD = filtered_Card_S[mask_diffOD_in_filtered].groupby('id_week_dest_dong')['netCardxorig_inf'].sum().reset_index()

    # Rename the column to reflect that it is an aggregated sum
    aggregated_Card_S_diffOD.rename(columns={'netCardxorig_inf': 'signetCardxinf_diff'}, inplace=True)

    # Create a decimal point adjusted-variable
    aggregated_Card_S_diffOD['signetCardxinf_diff_de4'] = aggregated_Card_S_diffOD['signetCardxinf_diff'] / 10000

    # want to add 'signetCardxinf_m1wk_diff_de4' column in df_input dataframe according to m1wk_dest
    df_input = pd.merge(df_input, aggregated_Card_S_diffOD[['id_week_dest_dong', 'signetCardxinf_diff_de4']],
        left_on='m1wk_dest', right_on='id_week_dest_dong', how='left')
    df_input['signetCardxinf_m1wk_diff_de4'] = df_input['signetCardxinf_diff_de4']
    df_input = df_input.drop(['id_week_dest_dong', 'signetCardxinf_diff_de4'], axis=1)

    # 5) 다른 OD간 카드사용 변화 관련 변수2 (나가는): 'Exposu_netCardxdestinf_wkm1_de3'
    if week == 20:
        # diffOD netCardxdest_inf 계산
        Card_S.loc[mask_diffOD, 'netCardxdest_inf'] = Card_S.loc[mask_diffOD, 'netCard'] * Card_S.loc[mask_diffOD, 'dest_inf']

        # Group by the 'id_week_orig_dong' and calculate sum of the 'netCard_diffxinfdest' values
        aggregated_Card_S_exposure = Card_S[mask_diffOD].groupby('id_week_orig_dong')['netCardxdest_inf'].sum().reset_index()
    else:
        # Update the Card_S['dest_inf'] from the predictions_df dataframe's week_{week - 1} column
        filtered_Card_S['dest_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['dest_dong_name'], m1wk_column]
            if row['dest_dong_name'] in predictions_df.index else row['dest_inf'], axis=1)

        # Update netCardxdest_inf
        filtered_Card_S.loc[:, 'netCardxdest_inf'] = filtered_Card_S['netCard'] * filtered_Card_S['dest_inf']

        # aggregate netCardxorig_inf by destination dong
        aggregated_Card_S_exposure = filtered_Card_S[mask_diffOD_in_filtered].groupby('id_week_orig_dong')['netCardxdest_inf'].sum().reset_index()

    aggregated_Card_S_exposure.rename(columns = {'netCardxdest_inf': 'signetCardxdest_inf'}, inplace = True)
    aggregated_Card_S_exposure['signetCardxdest_inf_de3'] = aggregated_Card_S_exposure['signetCardxdest_inf'] / 1000

    df_input = pd.merge(df_input, aggregated_Card_S_exposure[['id_week_orig_dong', 'signetCardxdest_inf_de3']],
        left_on = 'm1wk_dest', right_on = 'id_week_orig_dong', how = 'left')
    df_input['Exposu_netCardxdestinf_wkm1_de3'] = df_input['signetCardxdest_inf_de3']
    df_input = df_input.drop(['id_week_orig_dong', 'signetCardxdest_inf_de3'], axis = 1)

    # Replace NaN values with 0 in df_input
    df_input = df_input.fillna(0)

    # Save the current week's prediction input DataFrame to an Excel file
    df_input['dong_name'] = df_input.index
    df_input.to_excel(f'/content/drive/MyDrive/SP/input tables/df_input_week{week}.xlsx')

    # Predict the infection rates for the current week using the model
    X_input = df_input[['Infm2wk_orig_assumption',
                        'sigSWM_m1wkxinf_m1wk_de6',
                        'signetCardxinf_m1wk_same_de4',
                        'signetCardxinf_m1wk_diff_de4',
                        'Exposu_netCardxdestinf_wkm1_de3']]

    # Use the model to predict the infection rate
    y_pred = model.predict(X_input)

    # Replace negative values in y_pred with 0
    y_pred = np.where(y_pred < 0, 0, y_pred)

    # Update the predictions DataFrame with the predicted values
    predictions_df[f'week_{week}'] = y_pred

# Print the final predictions DataFrame
predictions_df.to_excel(f'/content/drive/MyDrive/SP/predictions_df.xlsx', index=True)
print(predictions_df)

<ipython-input-67-60b94915c8e6>:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_Card_S['dest_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['dest_dong_name'], m1wk_column]
<ipython-input-67-60b94915c8e6>:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_Card_S['dest_inf'] = filtered_Card_S.apply(lambda row: predictions_df.loc[row['dest_dong_name'], m1wk_column]
<ipython-input-67-60b94915c8e6>:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

           week_20   week_21   week_22   week_23   week_24   week_25  \
송파구가락1동   0.713043  0.709979  0.615650  0.925795  0.600964  0.763019   
송파구가락2동   1.135787  0.791523  1.033499  1.494823  1.236737  1.522667   
송파구가락본동   1.010740  0.817612  0.942501  1.264924  0.924952  1.375728   
구로구가리봉동   0.270650  0.071805  0.335254  0.784711  0.754199  0.952301   
금천구가산동    1.016545  0.776658  1.016367  2.109718  1.596464  2.480729   
...            ...       ...       ...       ...       ...       ...   
용산구효창동    0.549222  0.151343  0.171064  0.254025  0.138495  0.156843   
용산구후암동    1.463835  0.378050  0.418063  0.438236  0.000000  0.536559   
동대문구휘경1동  0.716863  0.513313  0.612301  0.924190  0.790699  1.181095   
동대문구휘경2동  0.959445  0.738794  0.853938  1.356971  0.944741  1.640767   
동작구흑석동    1.197715  1.219709  1.529742  2.052386  1.654600  2.321994   

           week_26   week_27   week_28   week_29    week_30    week_31  \
송파구가락1동   0.719589  0.525520  0.842605  4.667299   3.298964  